In [1]:
import pandas
import numpy
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
import time
import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# I - Градиентный бустинг

### 1. Загружаем данные

In [28]:
data_train = pandas.read_csv('C:/Data_P/Week_7/features_train.csv', index_col=False)
y_train = data_train['radiant_win']
X_train = data_train.loc[:, 'start_time':'dire_first_ward_time']

### 2. Проверка на наличие пропусков

In [33]:
X_train.count().tail(25)

d5_gold                        97230
d5_lh                          97230
d5_kills                       97230
d5_deaths                      97230
d5_items                       97230
first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_tpscroll_count         97230
radiant_boots_count            97230
radiant_ward_observer_count    97230
radiant_ward_sentry_count      97230
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_tpscroll_count            97230
dire_boots_count               97230
dire_ward_observer_count       97230
dire_ward_sentry_count         97230
dire_first_ward_time           95404
dtype: int64

Признаки, в которых имеются пропуски: first_blood_time, first_blood_team, first_blood_player1, first_blood_player2, radiant_bottle_time, radiant_courier_time, radiant_flying_courier_time, radiant_first_ward_time, dire_bottle_time, dire_courier_time, dire_flying_courier_time, dire_first_ward_time.
Как видно, имеется как минимум 2 признака, в которых существуют пропуски:
  - dire_first_ward_time - пропуски могут объясняться тем, что команда dire в первые 5 минут игры не установила предмет, который позволяет видеть часть игрового поля;
  - dire_flying_courier_time - пропуски могут объясняться тем, что команда dire в первые 5 минут игры не приобрела предмет "flying_courier".

### 3. Замена пропусков на нули

In [36]:
X_train = X_train.fillna(value=0)
X_train.count().tail(25)

d5_gold                        97230
d5_lh                          97230
d5_kills                       97230
d5_deaths                      97230
d5_items                       97230
first_blood_time               97230
first_blood_team               97230
first_blood_player1            97230
first_blood_player2            97230
radiant_bottle_time            97230
radiant_courier_time           97230
radiant_flying_courier_time    97230
radiant_tpscroll_count         97230
radiant_boots_count            97230
radiant_ward_observer_count    97230
radiant_ward_sentry_count      97230
radiant_first_ward_time        97230
dire_bottle_time               97230
dire_courier_time              97230
dire_flying_courier_time       97230
dire_tpscroll_count            97230
dire_boots_count               97230
dire_ward_observer_count       97230
dire_ward_sentry_count         97230
dire_first_ward_time           97230
dtype: int64

### 4. Зависимая переменная

В качестве зависимой переменной выступает radiant_win из data_train. В первом пункте она была помещена в y_train.

### 5. Обучение градиентного бустинга

Ниже приведен код функции, которая оценивает GBM с различным количеством деревьев в композиции (i * 10), вычисляет ROC_AUC для каждой композиции, а также время выполнения процесса для соответствующего количества деревьев. При оценке качества была использована кросс-валидация по 5 блокам.

In [37]:
def cross_GBM(X, y):
    ROC_AUC_GBM = pandas.DataFrame(columns=['n_estimators', 'ROC_AUC', 'time']) # Создали таблицу для занесения результатов.
    for i in range(1,6):
        model_GBM = GradientBoostingClassifier(n_estimators= i * 10, verbose=True, random_state=42) # Модель GBM с гиперпараметрами.
        start_time = datetime.datetime.now() # Время начала процесса кросс-валидации.
        cv = KFold(n_splits=5, shuffle=True, random_state=42) # Параметры кросс-валидации.
        score = cross_val_score(model_GBM, X, y, cv=cv, scoring='roc_auc').mean() # Обучение модели, и оценка ее качества через ROC_AUC с заданными параметрами кросс-валидации.
        time = datetime.datetime.now() - start_time # Вычисление времени процесса кросс-валидации.
        ROC_AUC_GBM.loc[i] = [i * 10, score, time] # Занесение результатов в таблицу для композиции с текущим количеством деревьев.
    return ROC_AUC_GBM # Возвращает таблицу с результатами.

Используя данную функцию, обучим градиентный бустинг по нашей выборке, а также оценим его качество при различном количестве решающих деревьев в композиции.

In [38]:
ROC_AUC_GBM = cross_GBM(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1           1.3785            7.33s
         2           1.3730            6.66s
         3           1.3679            6.95s
         4           1.3633            6.36s
         5           1.3588            5.09s
         6           1.3541            3.93s
         7           1.3497            3.07s
         8           1.3456            2.14s
         9           1.3415            1.07s
        10           1.3376            0.00s
      Iter       Train Loss   Remaining Time 
         1           1.3786           13.67s
         2           1.3732           12.35s
         3           1.3681           12.01s
         4           1.3638           10.83s
         5           1.3589            8.17s
         6           1.3546            6.33s
         7           1.3501            4.68s
         8           1.3458            2.97s
         9           1.3419            1.44s
        10           1.3381            0.00s
      It

In [39]:
ROC_AUC_GBM

,n_estimators,ROC_AUC,time
1,10,0.664851,00:01:04.332237
2,20,0.682462,00:02:24.632957
3,30,0.690006,00:03:09.334753
4,40,0.694039,00:04:59.388477
5,50,0.697494,00:05:29.757229


Как видно из таблицы, оптимум при количестве деревьев, равное 30, достигнут не был. При n_estimators = 30 ROC_AUC = 0.69, в то время как при n_estimators = 50 ROC_AUC = 0.697494. При этом есть предположение, с ростом числа деревьев качество будет расти (если судить о том, что при переходе с 30 к 40 и с 40 к 50 ROC_AUC растет). Так как качество растет, то имеет смысл и дальше увеличивать количество деревьев. Но затраты времени на кросс-валидацию существенны: для 30 деревьев затраты времени составили 3 минуты 10 секунд. И с ростом числа деревьев затраты времени также растут. Стоит отметить, что для ускорения обучения можно использовать не всю выборку, а какую то ее часть.

# II - Логистическая регрессия

### 1. Нормализация признаков и оценка логистической регрессии

Поскольку линейные алгоритмы чувствительны к масштабу признаков, последние были нормализованы.

In [40]:
X_train_standard = pandas.DataFrame(StandardScaler().fit_transform(X_train), index=X_train.index, columns=X_train.columns)

Ниже приведен код функции, которая оценивает логистичекую регрессию с различными параметрами L-2 регуляризации (параметры C), вычисляет ROC_AUC для каждого параметра, а также время выполнения процесса для соответствующего значения C. При оценке качества была использована кросс-валидация по 5 блокам.

In [41]:
def cross_logit(X, y):
    ROC_AUC_logit = pandas.DataFrame(columns=['C', 'ROC_AUC', 'time']) # Создали таблицу для занесения результатов.
    for i in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]:
        model_logit = LogisticRegression(C=i, random_state=42) # Модель logit с параметрами.
        start_time = datetime.datetime.now() # Время начала процесса кросс-валидации.
        cv = KFold(n_splits=5, shuffle=True, random_state=42) # Параметры кросс-валидации.
        score = cross_val_score(model_logit, X, y, cv=cv, scoring='roc_auc').mean() # Обучение модели, и оценка ее качества через ROC_AUC с заданными параметрами кросс-валидации.
        time = datetime.datetime.now() - start_time # Вычисление времени процесса кросс-валидации.
        ROC_AUC_logit.loc[i] = [i, score, time] # Занесение результатов в таблицу для текущего значения C.
    return ROC_AUC_logit # Возвращает таблицу с результатами.

Ниже представлена таблица с оценкой качества логистических регрессий с различными параметрами регуляризации:

In [42]:
ROC_AUC_logit = cross_logit(X_train_standard, y_train)

In [43]:
ROC_AUC_logit

,C,ROC_AUC,time
0.0001,0.0001,0.711352,00:00:01.602573
0.0010,0.0010,0.716363,00:00:02.526169
0.0100,0.0100,0.716550,00:00:03.476163
0.1000,0.1000,0.716527,00:00:04.039350
1.0000,1.0000,0.716522,00:00:03.441895
10.0000,10.0000,0.716522,00:00:03.813087
100.0000,100.0000,0.716522,00:00:04.775225
1000.0000,1000.0000,0.716522,00:00:05.115544


Из таблицы видно, что наибольшее значение ROC_AUC достигается при C = 0.1 (0.716527). Кроме того, скорость оценки в этом случае гораздо быстрее (4 секунды), чем в градиентном бустинге для 30 деревьев (3 минуты 10 секунд), а значение ROC_AUC выше. Это может объясняться тем, что связь между признаками и зависимой является линейной, а значит - вполне подойдет линейный алгоритм (логит), который будет давать качество не хуже нелинейного (градиентного бустинга), при этом будет быстрее обучаться.

### 2. Оценка логистической регресси по выборке без категориальных признаков

Поскольку в выобрке имеются категориальные признаки, а используются они как числовые, полезно было бы их выкинуть и снова оценить логистические регрессии.

In [44]:
X_train_standard = X_train_standard.drop(['lobby_type' ,'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis = 1)

In [45]:
ROC_AUC_logit_cat = cross_logit(X_train_standard, y_train)

In [46]:
ROC_AUC_logit_cat

,C,ROC_AUC,time
0.0001,0.0001,0.711333,00:00:02.597789
0.0010,0.0010,0.716376,00:00:04.350406
0.0100,0.0100,0.716559,00:00:05.215942
0.1000,0.1000,0.716534,00:00:05.669224
1.0000,1.0000,0.716530,00:00:05.628890
10.0000,10.0000,0.716531,00:00:05.607269
100.0000,100.0000,0.716530,00:00:05.611569
1000.0000,1000.0000,0.716530,00:00:05.642291


Как видно из таблицы, наибольшее значение ROC_AUC достигается при C = 0.01 (0.716559). При этом, качество текущей модели выше, чем в случае, когда модель оценивается по выборке с категориальными переменными. Это связано с тем, что в предыдущем случае категориальные признаки считались как числовые, что создавало определенные шумы и искажения.

### 3. Герои

Посчитаем количество уникальных героев в выборке, а также их возможное максимальное количество по максимальному id.

In [47]:
X_train_hero = X_train.loc[:, ['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']]
hero = X_train_hero.stack() # Сложили столбцы в строки.
len(hero.value_counts())

108

In [48]:
max(hero)

112

Таким образом, в нашей выборке 108 уникальных героев, а максимальный id героя - 112.

### 4. Создание мешка слов и добавление перекодированных категориальных признаков к числовым

In [49]:
X_pick = numpy.zeros((data_train.shape[0], 112))
for i, match_id in enumerate(data_train.index):
    for p in range(5):
        X_pick[i, data_train.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data_train.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

X_train_end = pandas.concat([X_train_standard, pandas.DataFrame(X_pick)],axis=1)

### 5. Оценивание логистической регрессии по выборке с перекодированными категориальными признаками

In [25]:
ROC_AUC_logit_end = cross_logit(X_train_end, y_train)

C:\Users\umche\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\umche\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

In [26]:
ROC_AUC_logit_end

,C,ROC_AUC,time
0.0001,0.0001,0.725038,00:00:02.926063
0.0010,0.0010,0.746333,00:00:05.126191
0.0100,0.0100,0.751739,00:00:09.418860
0.1000,0.1000,0.751946,00:00:14.362904
1.0000,1.0000,0.751927,00:00:12.516947
10.0000,10.0000,0.751926,00:00:14.437697
100.0000,100.0000,0.751923,00:00:13.872602
1000.0000,1000.0000,0.751922,00:00:15.201348


Как видно из таблицы, наибольшее значение ROC_AUC достигается при C = 0.1 (0.751946). Результат существенно улучшился. Это объясняется правильным включением категориальных признаков в обучаемую выборку.

### 6. Предсказание вероятностей по тестовой выборке с помощью лучшей модели

Загрузим тестовую выборку:

In [50]:
data_test = pandas.read_csv('C:/Data_P/Week_7/features_test.csv', index_col=False)

Поскольку лучшей моделью оказалась логистическая регрессия с параметром регуляризации C = 0.1 и оцененной по выборке с перекодированными категориальными признаками, нам необходимо осуществить обработку данных и для тестовой выборки. Другими словами, наобходимо заменить пропуски на нули, а также перекодировать категориальные признаки.

In [51]:
X_test = data_test.loc[:, 'start_time':'dire_first_ward_time']
X_test = X_test.fillna(value=0)
X_test_standard = pandas.DataFrame(StandardScaler().fit_transform(X_test), index=X_test.index, columns=X_test.columns)
X_test_standard = X_test_standard.drop(['lobby_type' ,'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis = 1)
X_test_hero = X_test.loc[:, ['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']]
hero_test = X_test_hero.stack()
X_pick_test = numpy.zeros((data_test.shape[0], max(hero_test)))
for i_test, match_id_test in enumerate(data_test.index):
    for p_test in range(5):
        X_pick_test[i_test, data_test.loc[match_id_test, 'r%d_hero' % (p_test+1)]-1] = 1
        X_pick_test[i_test, data_test.loc[match_id_test, 'd%d_hero' % (p_test+1)]-1] = -1
X_test_end = pandas.concat([X_test_standard, pandas.DataFrame(X_pick_test)],axis=1)

X_test_end содержит в себе тестовую выборку с готовыми данными. Предскажем с помощью логистической регрессии с C = 0.1 вероятности победы команды Rediant. 

In [53]:
model_best = LogisticRegression(C=0.1, random_state=42)
model_best.fit(X_train_end, y_train)

C:\Users\umche\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=0.1, random_state=42)

In [54]:
y_pred_test = model_best.predict_proba(X_test_end)

In [56]:
y_pred_test

array([[0.17539783, 0.82460217],
       [0.24271956, 0.75728044],
       [0.81248568, 0.18751432],
       ...,
       [0.76589828, 0.23410172],
       [0.37526772, 0.62473228],
       [0.57281178, 0.42718822]])

Как видно, вероятности от наблюдения к наблюдению отличаются. Другими словами, модель не является константной.

In [59]:
min(y_pred_test[0])

0.17539782679878269

In [60]:
max(y_pred_test[0])

0.8246021732012173

Предсказанные вероятности адекватны, поскольку минимальное и максимальное значение находятся в диапазоне [0:1]. И, соответственно, их сумма равна 1 по определению.